In [2]:
import pandas as pd
import cvxpy as cp
import numpy as np
import docplex.mp.model as md


# 加载数据


In [2]:
phase = np.loadtxt("phase")
phase=phase/phase[0].sum()
vol = np.loadtxt("volume")
qv = np.loadtxt("path_vol")
phase_get = np.load("phase_get.npy")
d = np.loadtxt("distance")
dr = np.loadtxt("dr")
dr=dr/phase[0].sum()
straight_get = np.load("straight_get.npy")
ison = np.array([0, 0, 0, 1, 1, 1])

px = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], ])


## 速度数据


In [3]:
lower, upper = 6.5, 13
linspace = np.arange(lower, upper, 0.5)
lin_num = len(linspace)

# 南北直行方向绿灯时间
sg=np.array([(straight_get[i]*phase).sum(axis=1) for i in range(len(straight_get))])
sr=1-sg

## 长干道数据


In [4]:
rho = vol[0] / vol[1]
num = len(vol[0])
numr = len(qv)
nump = len(phase_get)
M = 100000000
nx = 0.00000001
be = 8
speedScope = np.array([lower, upper])
speedVar = np.array([-2, 2])

g=np.array([(phase_get[i]*phase).sum(axis=1) for i in range(numr) ])

r=1-g

# 定义变量


In [5]:
cons = []


### 通用变量

In [6]:
z = cp.Variable(num)
cons += [z >= 1/180, z <= 1/80]
o = cp.Variable(num)
cons += [o >= 0, o <= 1]
t = cp.Variable((2, num-1))
k = 1

### 速度波动变量

In [7]:
yp = np.array([cp.Variable((num, lin_num), boolean=True) for i in range(2)])
nt = np.array([cp.Variable((num, lin_num), integer=True) for i in range(2)])
cons += [nt[i] >= -1 for i in range(2)]
cons += [nt[i] <= 10 for i in range(2)]


### 长干道变量

In [8]:
p = cp.Variable(num, boolean=True)
w = cp.Variable((numr, num))
cons += [w >= 0, w <= 1]
b = cp.Variable((numr, num))
cons += [b >= 0, b <= 1]
n = cp.Variable((numr, num), integer=True)
cons += [n >= 0, n <= 10]
u = cp.Variable((numr, num))
cons += [u >= 0, u <= 1]
y = cp.Variable((numr, num), boolean=True)

# 建立约束

## 长干道约束

保障路径$i$的绿波带宽出现出现在交叉口$k$内：
$$
\begin{cases}
b_{i,k}/2+\Delta_k\leq w_{i,k}\leq 1-r_{i,k}-b_{i,k}/2\\
\bar{b}_{i,k}/2+\bar{\Delta}_k\leq \bar{w}_{i,k} \leq 1-r_{i,k}-\bar{b}_{i,k}/2
\end{cases}
$$


In [9]:
cons+=[b/2+px<=w,w<=1-r-b/2]

速度变化区间为：
$$
\begin{cases}
\frac{d_k}{f_k}z\leq t_k \leq \frac{d_k}{e_k}z\\
\frac{\bar{d}_k}{\bar{f}_k}z \leq \bar{t}_k \leq \frac{\bar{d}_k}{\bar{e}_k}z\\
\frac{d_k}{h_k}z\leq \frac{d_k}{d_{k+1}}t_{k+1}-t_k \leq \frac{d_k}{g_k}z\\
\frac{\bar{d}_k}{\bar{h}_k}z \leq \frac{\bar{d}_k}{\bar{d}_{k+1}}\bar{t}_{k+1}-\bar{t}_k\leq \frac{\bar{d}_k}{\bar{g_k}}z
\end{cases}
$$

In [10]:
cons+=[cp.multiply(d/speedScope[1],z[:num-1])<=t[0],t[0]<=cp.multiply(d/speedScope[0],z[:num-1])]
cons+=[cp.multiply(d/speedScope[1],z[:num-1])<=t[1],t[1]<=cp.multiply(d/speedScope[0],z[:num-1])]

cons+=[cp.multiply(d[:num-2]/speedVar[0],z[:num-2])<=cp.multiply(d[:num-2]/d[1:],t[0,1:])-t[0,:num-2]]
cons+=[cp.multiply(d[:num-2]/d[1:],t[0,1:])-t[0,:num-2]<=cp.multiply(d[:num-2]/speedVar[1],z[:num-2])]

cons+=[cp.multiply(d[:num-2]/speedVar[0],z[:num-2])<=cp.multiply(d[:num-2]/d[1:],t[1,1:])-t[1,:num-2]]
cons+=[cp.multiply(d[:num-2]/d[1:],t[1,1:])-t[1,:num-2]<=cp.multiply(d[:num-2]/speedVar[1],z[:num-2])]

确保每个子区至少包含两个交叉口：
$$p_k+p_{k+1}<=1$$

In [11]:
cons+=[p[1:]+p[:num-1]<=1]

交叉口偏移量$\mu_{i,k}$，表示当交叉口$k$为干道分割点时，路径$i$在交叉口的偏移量，即当$p_k=1$时，$\tau_{i,k}>0$,否则，$\mu_{i,k}=0$,约束表达为：
$$
\epsilon p_k\leq \mu_{i,k}\leq p_k
$$

当路径$i$在交叉口$k$取得非零带宽，则应满足如下约束:
$$\begin{cases}
beZ_k-M (1-y_{i,k})\leq b_{i,k}\leq y_{i,k} \\
beZ_k-M (1-\bar{y}_{i,k})\leq \bar{b}_{i,k}\leq \bar{y}_{i,k}
\end{cases}
$$

In [12]:
for i in range(numr):
    cons+=[nx*p<=u[i],u[i]<=p]
    cons+=[be*z-M*(1-y[i])<=b[i],b[i]<=p]

相同分段子区内部的交叉口绿波带宽相同，约束表达为：
$$\begin{cases}
-M p_{k+1}\leq b_{i,k}-b_{i,k+1}\leq M p_{k+1}\\
-M p_{k}\leq \bar{b}_{i,k}-\bar{b}_{i,k+1}\leq M p_{k}
\end{cases}
$$

为避免路径产生无效带宽，引入二进制变量$y_{i,k}$,其定义如下：
$$
y_{i,k}=\begin{cases}
1,&\text{如果路径$i$在交叉口$k$能够获得有效非零带宽}\\
0,& \text{其他}
\end{cases}
$$

路径$i$在相同分组内取得相等带宽，进而约束相同分组内交叉口同时取得非零带宽，在不同分组可能无法取得带宽，其约束表达如下：

$$\begin{cases}
-M p_{k+1}\leq y_{i,k}-y_{i,k+1}\leq M p_{k+1}\\
-M p_{k}\leq \bar{y}_{i,k}-\bar{y}_{i,k+1}\leq M p_{k}
\end{cases}
$$

其含义为，当路径$i$在交叉口$k$能够取得非零带宽,则其带宽应大于基础带宽$be$

因引入$y_{i,k}$，当路径$i$在交叉口$k$无法取得有效带宽时，其循环整形约束应不再起作用，循环整形约束可修改为:

$$
\begin{cases}
\theta_k+r_{i,k}+\varphi_{i,k}+w_{i,k}+t_{k}+\mu_{k+1}\geq \theta_{k+1}+r_{i,k+1}+\varphi_{i,k+1}+w_{i,k+1}+n_{i,k+1}-M(1-y_{i,k})\\
\theta_k+r_{i,k}+\varphi_{i,k}+w_{i,k}+t_{k}+\mu_{k+1}\leq \theta_{k+1}+r_{i,k+1}+\varphi_{i,k+1}+w_{i,k+1}+n_{i,k+1}+M(1-y_{i,k})\\ 
\end{cases}
$$

$$
\begin{cases}
\theta_k+\bar{r}_{i,k}+\bar{\varphi}_{i,k}+\bar{w}_{i,k}+\bar{n}_{i,k}\geq\theta_{k+1}+\bar{r}_{i,k+1}+\bar{\varphi}_{i,k+1}+\bar{w}_{i,k+1}+\bar{t}_{k}+\bar{\mu}_{k}-M(1-y_{i,k})\\
\theta_k+\bar{r}_{i,k}+\bar{\varphi}_{i,k}+\bar{w}_{i,k}+\bar{n}_{i,k}\leq\theta_{k+1}+\bar{r}_{i,k+1}+\bar{\varphi}_{i,k+1}+\bar{w}_{i,k+1}+\bar{t}_{k}+\bar{\mu}_{k}+M(1-y_{i,k})
\end{cases}
$$

In [13]:
def add_split_on_cons(i,cons):
    cons += [-M*p[1:] <= b[i, :num-1]-b[i, 1:],
             b[i, :num-1]-b[i, 1:] <= M*p[1:]]

    cons += [-M*p[1:] <= y[i, :num-1]-y[i, 1:],
             y[i, :num-1]-y[i, 1:] <= M*p[1:]]

    cons += [o[:num-1]+r[i, :num-1]+dr[i, :num-1]+w[i, :num-1]+t[0, :num-1]+u[i, 1:] >=
             o[1:]+r[i, 1:]+dr[i, 1:]+w[i, 1:]+n[i, 1:]-M*(1-y[i, :num-1])]
    cons += [o[:num-1]+r[i, :num-1]+dr[i, :num-1]+w[i, :num-1]+t[0, :num-1]+u[i, 1:] <=
             o[1:]+r[i, 1:]+dr[i, 1:]+w[i, 1:]+n[i, 1:]+M*(1-y[i, :num-1])]


def add_split_in_cons(i,cons):
    cons += [-M*p[:num-1] <= b[i, :num-1]-b[i, 1:],
             b[i, :num-1]-b[i, 1:] <= M*p[:num-1]]

    cons += [-M*p[:num-1] <= y[i, 1:]-y[i, :num-1],
             y[i, 1:]-y[i, :num-1] <= M*p[:num-1]]

    cons += [o[:num-1]+r[i, :num-1]+dr[i, :num-1]+w[i, :num-1]+n[i, :num-1] >=
             o[1:]+r[i, 1:]+dr[i, 1:]+w[i, 1:]+t[1, :num-1]+u[i, :num-1]-M*(1-y[i, :num-1])]
    cons += [o[:num-1]+r[i, :num-1]+dr[i, :num-1]+w[i, :num-1]+n[i, :num-1] >=
             o[1:]+r[i, 1:]+dr[i, 1:]+w[i, 1:]+t[1, :num-1]+u[i, :num-1]-M*(1-y[i, :num-1])]


In [14]:
for i in range(numr):
    if ison[i]==0:
        add_split_on_cons(i,cons)
    elif ison[i]==1:
        add_split_in_cons(i,cons)

注意到某些情况下，模型可能更偏好某一方向带宽，从而将其他方向的带宽设置为0，为避免这种情况的发生，同时体现模型对上行（下行）方向的偏好，定义约束：

$$
(1-\rho_{k})\sum_i b_{i,k}\geq (1-\rho_{k})\rho_{k} \sum_i \bar{b}_{i,k}
$$

In [15]:
for k in range(num):
    cons+=[(1-rho[k-1])*cp.sum([b[i,k]*(1-ison[i]) for i in range(numr)])>=\
        (1-rho[k-1])*rho[k-1]*cp.sum([b[i,k]*ison[i] for i in range(numr)])]

因干道分割可对长干道进行分组，组内实现干道协调，不同组之间互不影响，因此不同分组可采用不同的信号协调周期，其约束表达如下：

$$
\begin{cases}
Z_{min}\leq Z_k\leq Z_{max}\\
-Mp_{k+1}\leq Z_{k+1}-Z_{k}\leq MP_{k+1}
\end{cases}
$$

In [16]:
cons+=[-M*p[1:]<=z[1:]-z[:num-1],z[1:]-z[:num-1]<=M*p[1:]]

通过该约束，可实现在不同分段子区之间，可采用不同的信号周期，避免过长的干道采用相同信号周期造成部分交叉口车辆延误过大

## 速度波动约束

插入图片

$i$表示路径，$j$表示相位，$k$表示交叉口
- $A_k$ ：表示上行带宽所处红灯结束（绿灯开始时间）点位

- $B_k$: 表示上行带宽所处绿灯结束（红灯开始时间）点位

- $A_{k}^{\prime}$:上行带宽与交叉口相位的左侧交点

- $B_{k}^{\prime}$:上行带宽与交叉口相位的右侧交点

- $\bar{A}_{k}$：表示下行带宽所处红灯结束（绿灯开始时间）点位

-  $\bar{B}_{k}$: 表示下行带宽所处绿灯结束（红灯开始时间）点位

- $\bar{A}_{k}^{\prime}$:下行带宽与交叉口相位的左侧交点

- $\bar{B}_{k}^{\prime}$:下行带宽与交叉口相位的右侧交点

$$\begin{cases}
A_{k,v}=\theta_k+rs_k\\
B_{k,v}=\theta_k+r_k+gs_k\\
A_{k,v}^{\prime}=max\{A_{k-1,v}^{\prime}+d_{k-1}/v*z_k-M*(1-p_k),A_{k,v}+\tau_k+n_{k,v}\}\\
B_{k,v}^{\prime}=min\{B_{k-1,v}^{\prime}+d_{k-1}/v*z_k+M*(1-p_k),B_{k,v}+n_{k,v}\}\\
\bar{A}_{k}=\theta_k+\bar{rs}_k+\Delta_k\\
\bar{B}_{k}=\theta_k+\bar{rs}_k+\bar{gs}_k+\Delta_k\\
\bar{A}_{k,v}^{\prime}=max\{\bar{A}_{k+1,v}^{\prime}+d_{k}/v*z_k-M*(1-p_k),\bar{A}_{k,v}+\bar{\tau}_k+\bar{n}_{k,v}\}\\
\bar{B}_{k,v}^{\prime}=min\{\bar{B}_{k+1,v}^{\prime}+d_{k}/v*z_k+M*(1-p_k),\bar{B}_{k,v}+\bar{n}_{k,v}\}
\end{cases}
$$


In [23]:
def add_var_on_cons(A, B, o, r, g, t, n, k, end, yp, px, p, sum_on, cons,rho):
    if k < end:
        A = cp.maximum(A+t[k-1]-M*(1-p[k]), o[k]+r[k]+n[k]+px[k])
        B = cp.minimum(B+t[k-1]+M*(1-p[k]), o[k]+r[k]+g[k]+n[k]+px[k])
        print(B-A)
        cons += [ B-A <= g[k]+M*(1-yp[k])]
        # cons += [-M*(1-yp[k]) <= B-A, B-A <= g[k]+M*(1-yp[k])]
        # cons += [be*z[k]-M*(1-yp[k]) <= B-A, B-A <= yp[k]+M*(1-yp[k])]
        # sum_on +=(B-A)*yp[k]
        return add_var_on_cons(A, B, o, r, g, t, n, k+1, end, yp, px, p, sum_on, cons,rho)
    else:
        return


# def add_var_in_cons(A, B, o, r, g, t, n, k, end, yp, px, p, sum_in, cons,rho):
#     if k > end:
#         A = cp.maximum(A+t[k]-M*(1-p[k]), o[k]+r[k]+n[k]+px[k])
#         B = cp.minimum(B+t[k]+M*(1-p[k]), o[k]+r[k]+g[k]+n[k])
#         cons += [-M*(1-yp[k]) <= B-A, B-A <= g[k]+M*(1-yp[k])]
#         cons += [be*z[k]-M*(1-yp[k]) <= B-A, B-A <= yp[k]+M*(1-yp[k])]
#         sum_in += (B-A)*yp[k]
#         return add_var_in_cons(A, B, o, r, g, t, n, k-1, end, yp, px, p, sum_in, cons,rho)
#     else:
#         return


In [24]:
sum_on=0
sum_in=0
for i,v in enumerate(linspace):
    add_var_on_cons(
        A=o[0]+sr[0,0],
        B=o[0]+sr[0,0]+sg[0,0],
        o=o,
        r=sr[0],
        g=sg[0],
        t=np.array([d[j]/v*z for j in range(num-1)]),
        n=np.array([nt[0][j,i] for j in range(num)]),
        k=1,
        end=num,
        yp=np.array([yp[0][j,i] for j in range(num)]),
        px=px[0],
        p=p,
        sum_on=sum_on,
        cons=cons,
        rho=rho
    )
    # add_var_in_cons(
    #     A=o[num-1]+sr[1,num-1],
    #     B=o[num-1]+sr[1,num-1]+sg[1,num-1],
    #     o=o,
    #     r=sr[1],
    #     g=sg[1],
    #     t=np.array([d[j]/v*z for j in range(num-1)]),
    #     n=np.array([nt[1][j,i] for j in range(num)]),
    #     k=num-2,
    #     end=-1,
    #     yp=np.array([yp[1][j,i] for j in range(num)]),
    #     px=px[1],
    #     p=p,
    #     sum_in=sum_in,
    #     cons=cons,
    #     rho=rho       
    # )

minimum(Promote(var12[0] + 0.5166666666666666 + 0.48333333333333334, (15,)) + Promote(36.30769230769231, (15,)) @ var1 + Promote(100000000.0 @ (1.0 + -var48[1]), (15,)), var12[1] + 0.6277777777777778 + 0.37222222222222223 + var26[1, 0]) + -maximum(Promote(var12[0] + 0.5166666666666666, (15,)) + Promote(36.30769230769231, (15,)) @ var1 + Promote(-100000000.0 @ (1.0 + -var48[1]), (15,)), var12[1] + 0.6277777777777778 + var26[1, 0])
minimum(minimum(Promote(var12[0] + 0.5166666666666666 + 0.48333333333333334, (15,)) + Promote(36.30769230769231, (15,)) @ var1 + Promote(100000000.0 @ (1.0 + -var48[1]), (15,)), var12[1] + 0.6277777777777778 + 0.37222222222222223 + var26[1, 0]) + Promote(70.3076923076923, (15,)) @ var1 + Promote(100000000.0 @ (1.0 + -var48[2]), (15,)), var12[2] + 0.6055555555555556 + 0.39444444444444443 + var26[2, 0]) + -maximum(maximum(Promote(var12[0] + 0.5166666666666666, (15,)) + Promote(36.30769230769231, (15,)) @ var1 + Promote(-100000000.0 @ (1.0 + -var48[1]), (15,)), v

为避免某些速度下无法取得有效带宽，进而影响其他速度下的带宽，可尽量
$$
\eta_{k,v}=\begin{cases}
1,&\text{如果速度$v$在交叉口$k$下能够获得有效带宽}\\
0,& \text{其他}
\end{cases}
$$
$$
\begin{cases}
be*z_k-M(1-\eta_{k,v})\leq B^{\prime}_{k,v} -A ^{\prime}_{k,v} \leq \eta_{k,v}+M*(1-\eta_{k,v})\\
be*z_k-M(1-\bar{\eta}_{k,v})\leq B^{\prime}_{k,v} -A ^{\prime}_{k,v} \leq \bar{\eta}_{k,v}+M*(1-\bar{\eta}_{k,v})
\end{cases}
$$

为了保证带宽式中为正，应确保点$B^{\prime}_k(\bar{B}^{\prime}_k)$始终处于点$A^{\prime}_k(\bar{A}^{\prime}_k)$的右侧，约束表示为：
$$
\begin{cases}
-M*(1-\eta_{k,v})\leq B^{\prime}_{k,v}-A^{\prime}_{k,v}\leq g_{k}+M*(1-\eta_{k,v})\\
-M*(1-\bar{\eta}_{k,v} )\leq\bar{B}^{\prime}_{k,v}-\bar{A}^{\prime}_{k,v} \leq \bar{g}_k+M*(1-\bar{\eta}_{k,v})
\end{cases}
$$

$$
\begin{cases}
b_{k,v}=B_{k,v}^{\prime}-A_{k,v}^{\prime},& k=1,2,...,n,&v \in \Omega_v \\
\bar{b}_{k,v}=\bar{B}_{k,v}^{\prime}-\bar{A}_{k,v}^{\prime},& k=0,1,...,n-1,&v \in \Omega_v
\end{cases}
$$

$$
(1-\rho_{k})\sum_i b_{k,v}\geq (1-\rho_{k})\rho_{k} \sum_i \bar{b}_{k,v}
$$

$$
\begin{cases}
-Mp_{k+1}\leq\eta_{k,v}-\eta_{k+1,v}\leq Mp_{k+1}\\
-Mp_{k}\leq\bar{\eta}_{k,v}-\bar{\eta}_{k+1,v}\leq Mp_{k}
\end{cases}
$$

In [19]:
cons+=[-M*p[k+1]<=yp[0][k,v]-yp[0][k+1,v] for k in range(num-1) for v in range(lin_num)]
cons+=[yp[0][k,v]-yp[0][k+1,v]<=M*p[k+1] for k in range(num-1) for v in range(lin_num)]

cons+=[-M*p[k]<=yp[1][k,v]-yp[1][k+1,v] for k in range(num-1) for v in range(lin_num)]
cons+=[yp[1][k,v]-yp[1][k+1,v]<=M*p[k] for k in range(num-1) for v in range(lin_num)]

# 构建目标函数

## 长干道目标构建

目标函数包括三部分：
- 最大化每条路径各交叉口的的绿波带宽。
- 最小化分割点停车等待时间。
- 最小化分割次数，并限制分割点流量。
$$
MAX:\sum_{i=1}^{I}\left(\omega_1\sum_{k=1}^{\Omega_i}\left(q_{i}b_{i,k}+\bar{q}_{i}\bar{b}_{i,k} \right)-\omega_2
\sum_{k=1}^{\Omega_i}\left(q_{i}\mu_{i,k}+\bar{q}_{i}\bar{\mu}_{i,k}\right)\right)-\omega_3\sum_{k=1}^{\Omega_i}\left(p_k\left(Q_k+\bar{Q}_k\right)\right)
$$

In [20]:
sum_b = cp.sum([qv[i] * b[i, k] for i in range(numr) for k in range(num)])
sum_u = cp.sum([qv[i] * u[i, k] for i in range(numr) for k in range(num)])
sum_p = cp.sum([p[k] * (vol[0, k] + vol[1, k]) for k in range(num)])
sum_split=sum_b*5+sum_u*4-sum_p

## 速度波动目标构建

$$
MAX: \sum_{v\in\Omega}\left(\omega_1\sum_{k=1}^{I}\eta_{k,v}(B^{\prime}_{k,v}-A^{\prime}_{k,v})+\bar{\eta}_{k,v}(\bar{B}^{\prime}_{k,v}-\bar{A}^{\prime}_{k,v})\right)
$$

In [21]:
sum_var=sum_in+sum_on

In [22]:
obj=cp.Maximize(sum_split+sum_var)
prob=cp.Problem(obj,cons)

prob.solve()
if prob.status!="optimal":
    print(prob.status)
print(prob.value)

DCPError: Problem does not follow DCP rules. Specifically:
The following constraints are not DCP:
minimum(Promote(var12[0] + 0.5166666666666666 + 0.48333333333333334, (15,)) + Promote(36.30769230769231, (15,)) @ var1 + Promote(100000000.0 @ (1.0 + -var48[1]), (15,)), var12[1] + 0.6277777777777778 + 0.37222222222222223 + var26[1, 0]) + -maximum(Promote(var12[0] + 0.5166666666666666, (15,)) + Promote(36.30769230769231, (15,)) @ var1 + Promote(-100000000.0 @ (1.0 + -var48[1]), (15,)), var12[1] + 0.6277777777777778 + var26[1, 0]) <= 0.37222222222222223 + 100000000.0 @ (1.0 + -var24[1, 0]) , because the following subexpressions are not:
|--  minimum(Promote(var12[0] + 0.5166666666666666 + 0.48333333333333334, (15,)) + Promote(36.30769230769231, (15,)) @ var1 + Promote(100000000.0 @ (1.0 + -var48[1]), (15,)), var12[1] + 0.6277777777777778 + 0.37222222222222223 + var26[1, 0]) + -maximum(Promote(var12[0] + 0.5166666666666666, (15,)) + Promote(36.30769230769231, (15,)) @ var1 + Promote(-1000000

In [ ]:
p=np.array(p.value)
o=np.array(o.value)
w=np.array(w.value)
n=np.array(n.value)
t=np.array(t.value)
u=np.array(u.value)
y=np.array(y.value)
z=np.array(z.value)
b=np.array(b.value)

In [65]:
M=1000000
a=cp.Variable()
b=cp.Variable()
c=cp.Variable()
d=cp.Variable()
p=cp.Variable(boolean=True)
cons=[a>=1,a<=4]
cons+=[b>=2,b<=4]
cons+=[c>=3,c<=4]
cons+=[d>=4,d<=5]
cons+=[min(a,2)<=2]
# A=a if a>=b else b
# B=c if c>=d else d
# cons+=[B-A<=5]
obj=cp.Maximize(a+b+c)
prob=cp.Problem(obj,cons)
prob.solve(solver=cp.CPLEX)
prob.status

NotImplementedError: Strict inequalities are not allowed.

In [60]:
a.value

In [52]:
import docplex.mp.model as md

model=md.Model()
a=model.continuous_var(lb=1,ub=4)
b=model.continuous_var(lb=2,ub=4)
c=model.continuous_var(lb=3,ub=4)
d=model.continuous_var(lb=4,ub=5)
A=model.max(a,b)
B=model.min(c,d)
model.add_constraint(B-A<=5)
model.maximize(a+b+c)
s=model.solve()
s.objective_value

12.0